# Facultad de Ingeniería Mecánica y Electrica
# Ingeniería en Computación Inteligente
# Redes Neuronales
# Profesor: Luis Eduardo Morán López
# 7 D

# Estudiantes:
## Cristian Armando Larios Bravo
## Hernández Paredes Roberto Alejandro
## Gabriel Alejandro Gudiño Méndez

# Red Neuronal: **RNC**
# Dataset: **Cards**


# Cristian

In [1]:
# Importacion de librerias
%matplotlib inline

# import matplotlib.pyplot as plt
import os
import torch
# from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from PIL import Image
from torch.nn import functional as Fun
import torch.nn as nn

In [20]:
class RNC(nn.Module):
  # Constructor
  def __init__(self, input_size, num_classes):
    super(RNC, self).__init__()

    self.conv1 = nn.Conv2d(in_channels=3, out_channels=224, kernel_size=3, stride=1, padding=1)
    self.pool = nn.MaxPool2d(kernel_size=2)
    self.conv2 = nn.Conv2d(in_channels=224, out_channels=224, kernel_size=3, stride=1, padding=1)
    self.conv3 = nn.Conv2d(in_channels=224, out_channels=224, kernel_size=3, stride=1, padding=1)
    # A drop layer deletes 20% of the features to help prevent overfitting
    self.drop = nn.Dropout2d(p=0.2)

    self.fc = nn.Linear(in_features=25*25*24, out_features=num_classes)

  def forward(self, x):
    x = Fun.relu(self.conv1(x))
    # x = self.pool(x)
    x = Fun.relu(self.conv2(x))
    # x = self.pool(x)
    x = Fun.relu(self.conv3(x))

    x = Fun.dropout(x, training=self.training)
    # Flatten
    x = x.view(-1, 25*25*24)
    # Feed to fully-connected layer to predict class
    x = self.fc(x)
    return Fun.log_softmax(x, dim=1)

In [16]:
def train(model: RNC, device, train_loader, optimizer, epoch):
  # Set model to training mode
  model.train()
  train_loss = 0
  print("Epoch:", epoch)

  # Process the image in batches
  for batch_idx, (data, target) in enumerate(train_loader):
    # Move the data to the selected device
    data, target = data.to(device), target.to(device)

    # Reset the optimizer
    optimizer.zero_grad()

    # Push the data forward through the model layers
    output = model(data)

    # Get the loss
    loss = Fun.nll_loss(output, target)

    # Keep a running total
    train_loss += loss.item()

    # Backpropagate
    loss.backward()
    optimizer.step()
    # train_loss += Fun.nll_loss(output, target, size_average=False).data.item()

    # return average loss for epoch
    avg_loss = train_loss / (batch_idx+1)
  print('\nTrain set: Average loss: {:.6f}'.format(avg_loss))
  return avg_loss

In [7]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define data transformations (if needed)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # Resize if necessary
    transforms.ToTensor(),
])

# Create datasets using ImageFolder
train_dataset = datasets.ImageFolder(root='Cards/Train', transform=data_transforms)
test_dataset = datasets.ImageFolder(root='Cards/Test', transform=data_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [21]:
modelo = RNC(input_size=224, num_classes=2)
print(modelo)

RNC(
  (conv1): Conv2d(3, 224, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(224, 224, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(224, 224, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (drop): Dropout2d(p=0.2, inplace=False)
  (fc): Linear(in_features=15000, out_features=2, bias=True)
)


In [18]:
avg_loss = train(modelo, 'cpu', train_loader, optim.Adam(modelo.parameters(), lr=0.001), 1)

Epoch: 1


RuntimeError: shape '[-1, 15000]' is invalid for input of size 359661568

# Otro

# Otro 2

# PRUEBAS

In [ ]:
%matplotlib inline
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            # 1st Layer to 1st Hidden Layer
            nn.Linear(28*28, 512), # 28*28 input layer to 512 features
            nn.ReLU(),
            # 1st Hidden Layer to 2nd Hidden Layer
            nn.Linear(512, 512), # 512 input and transforms to next hidden layer 512 features
            nn.ReLU(),
            # 2nd Hidden Layer to Output Layer
            nn.Linear(512, 10), # 512 input to 10 (number of classes)
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1])


In [ ]:
print(f"First Linear weights: {model.linear_relu_stack[0].weight} \n")

print(f"First Linear biases: {model.linear_relu_stack[0].bias} \n")

First Linear weights: Parameter containing:
tensor([[-0.0275,  0.0216, -0.0241,  ..., -0.0271,  0.0080, -0.0313],
        [ 0.0344,  0.0288, -0.0202,  ...,  0.0297,  0.0296,  0.0066],
        [ 0.0244,  0.0267,  0.0134,  ..., -0.0333,  0.0010,  0.0194],
        ...,
        [ 0.0230, -0.0057, -0.0214,  ..., -0.0220,  0.0329, -0.0008],
        [ 0.0288, -0.0021, -0.0085,  ...,  0.0023, -0.0073, -0.0184],
        [-0.0004, -0.0068, -0.0165,  ..., -0.0279, -0.0090, -0.0260]],
       requires_grad=True) 

First Linear biases: Parameter containing:
tensor([ 1.4137e-03, -2.2607e-02, -6.9330e-03, -1.8913e-02,  2.5496e-02,
        -2.2059e-02,  1.7675e-03, -1.2923e-02, -2.5342e-02, -2.6902e-02,
        -2.0974e-02,  8.7219e-05, -5.6567e-03, -3.0013e-02,  1.2608e-02,
         2.6354e-02,  2.0680e-02,  4.8043e-03, -2.2867e-02,  8.5346e-03,
         6.8256e-03, -2.7548e-02,  8.4559e-03, -2.5671e-02,  1.2962e-02,
         3.3586e-02, -8.8344e-03, -1.3171e-02,  1.7949e-02, -1.9422e-02,
        -2.3

In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.6771, -0.1177,  0.0798, -0.0326,  0.3827, -0.0638, -0.0817, -0.1258,
          0.1947, -0.2940, -0.4268,  0.1580,  0.0095,  0.4722,  0.1491,  0.0661,
         -0.0646, -1.0165, -0.0515,  0.4274],
        [ 0.3972, -0.2417,  0.3350,  0.2758,  0.2203,  0.4343,  0.1013, -0.1765,
          0.3746, -0.4195, -0.7141,  0.3570,  0.0453,  0.2282,  0.2823,  0.1632,
         -0.2641, -0.7487, -0.0686,  0.1702],
        [ 0.6783,  0.1336,  0.3200,  0.1256,  0.6135,  0.2040,  0.0309, -0.0824,
          0.6925, -0.1859, -0.3608, -0.0410,  0.0640,  0.6310,  0.3480,  0.3770,
         -0.0829, -0.8429, -0.4251, -0.2238]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.6771, 0.0000, 0.0798, 0.0000, 0.3827, 0.0000, 0.0000, 0.0000, 0.1947,
         0.0000, 0.0000, 0.1580, 0.0095, 0.4722, 0.1491, 0.0661, 0.0000, 0.0000,
         0.0000, 0.4274],
        [0.3972, 0.0000, 0.3350, 0.2758, 0.2203, 0.4343, 0.1013, 0.0000, 0.3746,
         0.0000, 0.0000, 0.3570, 0.0453, 0.2282, 0.28

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [ ]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0275,  0.0216, -0.0241,  ..., -0.0271,  0.0080, -0.0313],
        [ 0.0344,  0.0288, -0.0202,  ...,  0.0297,  0.0296,  0.0066]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0014, -0.0226], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0376,  0.0307, -0.0241,  ..., -0.0404,  0.0410,  0.0325],
        [ 0.0409, -0.0291, -0.0269,  ..., -0.0300, -0.0188,  0.0263]],
       grad_fn=<SliceBackward0>) 

Layer: linear_re